In [0]:
%pip install -q -U google-genai --quiet

In [0]:
from google import genai
from google.genai import types
from pyspark.sql import functions as F

# Get Gemini Key
GEMINI_API_KEY = dbutils.widgets.get("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

# 1) Pull latest gold row
gold = spark.table("travel_planning.gold.daily_context") \
            .orderBy(F.col("ingest_ts_nyc").desc()) \
            .limit(1)

row = gold.collect()[0]

# 2) Extract fields
city = row["city"]
country = row["country"]
weather = row["weather"]
temp = row["temp"]
tmin = row["tmin"]
tmax = row["tmax"]
sunrise = row["sunrise"]
sunset = row["sunset"]
restaurants_json = row["restaurants_json"]   # already a JSON array string
attractions_json = row["attractions_json"]
# 3) Build a better prompt for Gemini
system = """
You are a travel planner assistant.
Use the weather + restaurant list to recommend a plan.
Prefer higher rating + more reviews.
If it's cold, prioritize indoor spots.
Return concise bullet points.
"""

user_prompt = f"""
City: {city}, {country}
Weather: {weather}
Temp(F): now={temp}, min={tmin}, max={tmax}
Sunrise: {sunrise}, Sunset: {sunset}

Restaurants/places (JSON array):
{restaurants_json}
Attractions (JSON array):
{attractions_json}

Task:
- Recommend restaurants and places to travel, make it suitable.
- For each pick: give name + why (1 short reason) + address.
- If 'open_now' is false or missing for the time of the meal or the attraction, mention its time and remind the traveller about it.
"""

# 4) Call Gemini
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(system_instruction=system),
    contents=user_prompt
)

suggestion = response.text
print(suggestion)

In [0]:
out_df = gold.withColumn("gemini_plan", F.lit(suggestion))

(out_df
 .write
 .format("delta")
 .mode("append")
 .option("mergeSchema", "true")
 .saveAsTable("travel_planning.gold.recommendations"))